In [1]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob
import os
from os import *
import re
import imutils
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import collections
from ipynb.fs.full.Track_Functions import *

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]


In [3]:
p = '/home/jovyan/DATA_MASTER_PROJECT/Check_DIFF_T40_T70/full_size_CONTROLL/'
#p = '/home/jovyan/send_ola/'
#t = []
#for i in listdir(p):
    #im =cv2.imread(p+i)
    #if np.count_nonzero(im == 255) < 1000:
       # t.append(im)

In [ ]:
# for ls, i in enumerate(t):

#         im = Image.fromarray(i)

#         im.save('{0}/well_A_time_{1}_id.png'.format(via, ls))  

In [4]:
name_well = []
for di,dr,f in os.walk(p):
    for i in f:
        name_well.append(i.split(' ')[1].split(' ')[0])

In [5]:
wells = list(set(name_well))
wells

['B3-3',
 'B3-1',
 'B3-4',
 'D6-2',
 'F10-4',
 'D6-1',
 'D6-3',
 'F10-1',
 'D6-4',
 'F10-3',
 'B3-2',
 'F10-2']

In [21]:
count = 0
for di,dr,f in os.walk(p):
    for i in f: 
        if 'C4-2' in i: 
            count +=1

In [22]:
count

30

In [4]:
def get_time_point(path, time_point_1,time_point_2 ,time_point_3):   # time_point in form of 'Take #1 ' (remember the space between #1 ') or 'Take #32'

    image_time_point_1 = []
    image_time_point_2 = []
    image_time_point_3 = []

    for dirname, dirnames,filenames in os.walk(path):



        for filename in sorted(filenames, key = natural_keys):



            if time_point_1 in filename:
                
                im1 = cv2.imread(os.path.join(dirname, filename))
                if np.count_nonzero(im1 == 255) < 17777000:
                    image_time_point_1.append(im1)
                
            #elif time_point_2 in filename: 
                
                #im2 = cv2.imread(os.path.join(dirname, filename))
                #if np.count_nonzero(im2 == 255) < 177777000:
                    #image_time_point_2.append(im2)
                
            #elif time_point_3 in filename:
            
             #   im3 = cv2.imread(os.path.join(dirname, filename))
              #  if np.count_nonzero(im3 == 255) < 177777000:
               #     image_time_point_3.append(im3)
                    
            

    return image_time_point_1#, image_time_point_2, image_time_point_3 

In [5]:
t = get_time_point(p,'B3-3','D5-2','F10-2')

In [ ]:
def find_and_cut(path_img,time_point_1,time_point_2,time_point_3, save_path):

    img_list_1, img_list_2, img_list_3 = get_time_point(path_img,time_point_1,time_point_2,time_point_3)

    list_img = []
    list_img.append(img_list_1)
    #list_img.append(img_list_2)
    #list_img.append(img_list_3)
    
    wells = [time_point_1,time_point_2,time_point_3] 
    
    crop_count = []

    for list_ix, _ in enumerate(list_img):

        t = []
        t = list_img[list_ix]


        dict_cord = {}
        cor = []
        x = []
        y = []
        for i in tqdm.tqdm(t):
            cor_0,x0,y0 = return_cord(i)
            cor.append(cor_0)
            x.append(x0)
            y.append(y0)

            data={'X':x, 'Y':y, 'Cor':cor}
            data = pd.DataFrame(data)


        dict_cord = data
        coords = dict_cord.values.tolist()

        id_cell_pr = get_id_prime(coords[0][0],coords[0][1])


        rows_1_2, cols_1_2, list_id_index_1_2 ,d_1_2 = eucl_dist_matrix_t1(coords[0][2], coords[1][2])

        try:
            index, duplicate_columns = find_duplicates_col_t1(rows_1_2, cols_1_2, list_id_index_1_2, d_1_2)

            x_1, y_1, new_cor_1, id_cell_new = remove_duplicates_t1(coords[1][0],coords[1][1], index, id_cell_pr)

            new_coor = {'Cor_1':new_cor_1}

            new_id = {'ID_1':id_cell_new}

            for ix in range(len(t)):

                row, col, list_id_index1 ,d1 = eucl_dist_matrix_t1(list(new_coor.values())[ix], coords[2+ix][2])

                row1, col1 = zip(*list_id_index1)


                # Index duplicates cols


                duplicates2 = []
                for item in col1:
                    if col1.count(item) > 1:
                        duplicates2.append(item)
                dup_set2 = set(duplicates2)


                # Index unused cols

                r = len(list(new_coor.values())[ix])
                unused_col = list(range(0, r))
                unused = set(col1)
                idx_unused_col = [x for x in unused_col if x not in unused]


                n_cord_new, id_cell_new = final_fun(dup_set2, idx_unused_col, d1, list(new_id.values())[ix], list_id_index1, coords[2+ix][2], list(new_coor.values())[ix])

                new_coor['Cor_{}'.format(2+ix)] = n_cord_new
                new_id['ID_{}'.format(2+ix)] = id_cell_new


        except IndexError:
            pass


        del t[0]

        coor = []
        id_c = []

        for k, v in new_coor.items():
            coor.append(v)

        for p, c in new_id.items():
            id_c.append(c)

        for idx , _ in enumerate(t):

            img = t[idx]

            crop_cell_1 = []

            x = []
            y = []

            for ix, i in enumerate(coor[idx]):
                x.append(coor[idx][ix][0])      
                y.append(coor[idx][ix][1])


            w = 45
            h = 45

            try_id = []

            for ix, i in enumerate(x):

                s = id_c[idx][ix]


                crop = img[(y[ix]-h):(y[ix]+h), (x[ix]-w):(x[ix]+w)]

                if crop.size == 24300: 

                    crop_cell_1.append(crop) 
                    try_id.append(s)

            for ls, i in enumerate(crop_cell_1):

                im = Image.fromarray(i)

                im.save(save_path.format(via, wells[list_ix],try_id[ls], int(idx+1)))  
                
            crop_count.append(len(crop_cell_1))
            
    return sum(crop_count)


In [ ]:
via = '/home/jovyan/DATA_MASTER_PROJECT/cont_crop'
directory = os.mkdir(via) 

In [ ]:
p = '/mnt/holomonitor-pvc/finalprojecttest/'

In [ ]:
count = find_and_cut(p , 'D5-2','E4-3','E6-2' , '{0}/{1}_id_{2}_time_{3}.tiff')

count

In [ ]:
#im.save(via + wells[list_ix] + 't_{0}_id_{1}.png'.format(int(idx+1), try_id[ls]))  

In [ ]:
time = []

for dirname, dirnames,filenames in os.walk('/home/jovyan/MASTER_PROJECT_NOTEBOOKS/try_id_cell_cropped_id/'):
    
    for filename in sorted(filenames, key = natural_keys):
        if 'B3' in filename: #PAY ATTENTION ID THE IMAGE IS A TIFF OR PNG IMAGE #########
            time.append(filename)

len(time)

In [ ]:
#check how many diff ID and make al list of list based on number of id

data_id = {}
n_id = []

for i in time:
    t = i.split('_id_')[1].split('time_')[0]
    n_id.append(t)

id_cell = set(n_id)

for ix, i in enumerate(sorted(id_cell, key = natural_keys)):
    
    id_name = []
    dict_1 = {}
    
    for t in time:
        if 'id_{}'.format(i) in t:
            id_name.append(t)
    
    d = {'id':id_name}
    data = pd.DataFrame(d)

    dict_1[ix]=data 
    data_id.update(dict_1)       
            

In [ ]:
delete = [i for i, j in data_id.items() if len(j) < 9] # 9 or the length of time span you are traning on 

In [ ]:
for i in delete : del data_id[i]

In [ ]:
data_id

In [ ]:
image_list = []
path = '/home/jovyan/MASTER_PROJECT_NOTEBOOKS/try_id_cell_cropped_id/'
for ix , i in enumerate(time):
    for filename in glob.glob(path + i): 
        im=cv2.imread(filename)

        image_list.append(im)
len(image_list)

In [ ]:
plt.figure(figsize=(20,20))
for i in range(len(image_list)):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image_list[i], cmap=plt.cm.binary)
    #plt.xlabel(class_names[y_orig_1[i]])
plt.show()

In [ ]:
# def find_and_cut(path_img,time_point_1,time_point_2,time_point_3, save_path): #'Take #1 ', # "{0}/sk_{1}_exp_2_time_1.tiff"

#     t_1 = get_time_point(path_img,time_point_1,time_point_2,time_point_3)

#     tot_dict = {}
    
    

#     for idx, image in enumerate(t_1):

#         dict_1 = {}

#         cx = []
#         cy = []

#         shifted = cv2.pyrMeanShiftFiltering(image, 10, 50)
#         gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)
#         thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
#         D = ndimage.distance_transform_edt(thresh)
#         localMax = peak_local_max(D, indices=False, min_distance=9, labels=thresh)
#         markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
#         labels = watershed(-D, markers, mask=thresh)



#         for label in np.unique(labels):
#             if label == 0:
#                 continue




#             mask = np.zeros(gray.shape, dtype='uint8')
#             mask[labels == label] = 255
#             cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#             cnts = imutils.grab_contours(cnts)
#             c = max(cnts, key=cv2.contourArea)
#             ((x, y), r) = cv2.minEnclosingCircle(c)



#             cx.append(int(x))
#             cy.append(int(y))


#             data = {'X':cx,'Y':cy}

#             data = pd.DataFrame(data)


#         dict_1[idx] = data

#         tot_dict.update(dict_1)


#     t_1_1 = get_time_point(path_img, time_point_1,time_point_2,time_point_3)


#     value_dict = []

#     for k, v in sorted(tot_dict.items()):
#         value_dict.append(v)



#     crop_cell_1 = []

#     for idx , i in enumerate(t_1_1):

#         img = t_1_1[idx]



#         x = []
#         y = []

#         for i in range(len(value_dict[idx])):
#             x.append(value_dict[idx].iloc[i,0])      
#             y.append(value_dict[idx].iloc[i,1])


#         w = 45
#         h = 45

#         for ix, i in enumerate(x):

#             crop = img[(y[ix]-h):(y[ix]+h), (x[ix]-w):(x[ix]+w)]

#             if crop.size == 24300: 

#                 crop_cell_1.append(crop)





#     for ls, i in enumerate(crop_cell_1):

#         im = Image.fromarray(i)

#         im.save(save_path.format(via, ls))   # exp 1 or 2 sonce they did dublicate studies 

#     return crop_cell_1

In [ ]:
via = '/home/jovyan/DATA_MASTER_PROJECT/A549/Cropped_test'
directory = os.mkdir(via) 

In [ ]:
path_img = p


In [ ]:
croped_cell = find_and_cut(path_img, 'B3','D3','F3' , "{0}/D_2{1}_.tiff")

len(croped_cell)

In [ ]:


a0 = [['controll', 0]] * 8349

a1 =[['D_2', 1]] * 

a2 =[['H_2', 2]] * 7480

#a3 =[['mast', 3]] * 50

# a4 =[['misoprostol', 4]] * 8371

# a5 =[['molindone', 5]] * 6766

# a6 =[['nadol', 6]] * 7420

# a7 =[['nadol', 7]] * 7420



labels = a1+a0+a2#+a2#+a2+a3+a4+a5+a6



In [ ]:
df = pd.DataFrame(labels, columns = ['Type', 'Category'])
df = df.sort_values(by=['Type'])

In [ ]:

df.to_csv("/home/jovyan/DATA_MASTER_PROJECT/A549/test_labels.csv", sep=',',index=False)